In [102]:

from pulp import * 
import numpy as np
import pandas as pd
import math

members_count = 30
days_count = 26

members = list(range(1, members_count + 1))
days = list(range(1, days_count + 1))
shifts = list(range(1,5))
# defining the variables
x = LpVariable.dicts("x",(members,days,shifts),0,1,LpInteger)
y = LpVariable.dicts("y",(members),0,None,LpInteger)
# binary variable
b = LpVariable.dicts("b",(members),0,1,LpInteger)

model = LpProblem("Shifts", LpMaximize)
model += lpSum(y[si] for si in members)

# # 4 shifts per meeting
for d in days:
    model += lpSum(x[m][d][r] for m in members for r in shifts) == 4

# # max and min number of meetings per member
for m in members:
    model += lpSum(x[m][d][r] for d in days for r in shifts) <= math.ceil(days_count*4 / members_count)
    model += lpSum(x[m][d][r] for d in days for r in shifts) >= math.floor(days_count*4 / members_count)

# only one role per member per meeting
for m in members:
    for d in days:
        model += lpSum(x[m][d][r] for r in shifts) <= 1

# every role in the meeting must be occupied
for d in days:
    for r in shifts:
        model += lpSum(x[m][d][r] for m in members) == 1

# number of shifts that a member has done
for m in members:
    model += lpSum(x[m][d][r] for d in days for r in shifts) - y[m] == 0


model.solve()

1

In [99]:
days = [i for i in range(1, 26)]
shifts = [i for _ in days for i in range(1,5)]
shift_weights = [1.5 if i == 1 else 1 for i in shifts]
members = [i for i in range(1, 31)]

has_shift = LpVariable.dicts("hasshift", (members, shifts), 0, 1, LpInteger)
member_vars = LpVariable.dicts("membervars", (members), 0, None, LpInteger)
weight_vars = LpVariable.dicts("weightvars", (shift_weights), 0)

model = LpProblem("Shifts", LpMinimize)
model += lpSum(member_vars[m]*shift_weights[w] for m in members for w in shift_weights)

# # 4 shifts per meeting
model += lpSum(has_shift[m][r] for m in members for r in shifts) == 4

# only one role per member per meeting
for m in members:
    model += lpSum(has_shift[m][r] for r in shifts) <= 1

# every role in the meeting must be occupied
for r in shifts:
    model += lpSum(has_shift[m][r] for m in members) == 1

for m in members:
    model += lpSum(has_shift[m][r] for r in shifts) - member_vars[m] == 0


model.solve()



TypeError: list indices must be integers or slices, not float

In [103]:
people = []
days = []
shifts = []
value = []

for vi in model.variables():
    
    if vi.name.split("_")[0] == 'x':
    
        person = vi.name.split("_")[1]
        day = vi.name.split("_")[2]
        role = vi.name.split("_")[3]

        people.append(int(person))
        days.append(int(day))
        shifts.append(role)
        value.append(vi.varValue)


data = {'Person':people, 'Day':days, 'Role':shifts, 'Value':value} 

df = pd.DataFrame(data)
pd.set_option('display.max_columns', None)

def mapper(row):
    if row['Value'] == 1:
        return str(int(row['Role']))
    return str(int(row['Value']))

df['y'] = df.apply(mapper,axis = 1)

In [104]:

df['z'] = df['y'].apply(int)
df.sort_index()

table = pd.pivot_table(df, values='z', index=['Person'],
                    columns=['Day'], aggfunc=np.sum)

table

Day,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
Person,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,3,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,3,2,0,3,0,0,0,0
3,0,0,0,3,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0
5,4,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,1,0,0,0,0,0,2,0
7,0,1,0,2,0,0,0,0,0,0,4,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0
8,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,1,0,0,4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4
